Hello! Thank you for checking out our tool.

The purpose of this demo is demonstrate some of the basics. In doing so, we will generate a flipset for one individual. In doing so, we'll show:

1. How to use the ActionSet interface to specify immutable variables and variables with custom ranges.
2. How to use a model to align an ActionSet
3. How to use the RecourseBuilder interface to find the feasibility of one person.

We'll work using CPLEX. The problem is equivalent for CBC. To install either package, read [here](https://github.com/ustunb/actionable-recourse/blob/master/README.md).

In [18]:
import os
import numpy as np
import pandas as pd
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV
import sklearn
import sys
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)
from recourse.builder import RecourseBuilder
from recourse.builder import ActionSet
from recourse.flipset import Flipset

data_dir = "../data/2_1_experiment_1/"

# German Credit dataset

In [19]:
data_name = 'german_processed'
data_file = os.path.join(data_dir, '%s.csv' % data_name)
## load and process data
german_df = pd.read_csv(data_file).reset_index(drop=True)

german_df = (german_df
             .assign(isMale=lambda df: (df['Gender']=='Male').astype(int))
             .drop(['PurposeOfLoan', 'Gender', 'OtherLoansAtStore'], axis=1)
            )

german_y = german_df['GoodCustomer']
german_X = german_df.drop('GoodCustomer', axis=1)

german_categorical_features = [0, 1, 9, 10, 11, 12, 13, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26]
columns = german_X.columns
german_categorical_names = [columns[i] for i in german_categorical_features] 

# COMPAS dataset

In [20]:
data_name = "compas-scores-two-years"
data_file = os.path.join(data_dir, '%s.csv' % data_name)
## load and process data
compas_df = pd.read_csv(data_file).reset_index(drop=True)

cols_with_missing_values = []
for col in compas_df.columns:
    if len(np.where(compas_df[col].values == '?')[0]) >= 1 or compas_df[col].isnull().values.any():
        cols_with_missing_values.append(col)    

# compas_df['length_of_stay'] = (pd.to_datetime(compas_df['c_jail_out']) - pd.to_datetime(compas_df['c_jail_in'])).dt.days
compas_df = compas_df.drop(cols_with_missing_values, axis=1)
compas_df = (compas_df
             .drop(['id', 'name', 'first', 'last', 'dob', 'compas_screening_date', 'type_of_assessment', \
                    'screening_date', 'v_type_of_assessment', 'v_screening_date', 'race'], axis=1)
            )

compas_df = pd.get_dummies(compas_df, columns=['sex']).drop(['sex_Female'], axis=1)
# compas_df = pd.get_dummies(compas_df, columns=['race'])
compas_df = pd.get_dummies(compas_df, columns=['age_cat'])
compas_df = pd.get_dummies(compas_df, columns=['score_text'])
compas_df = pd.get_dummies(compas_df, columns=['v_score_text'])
compas_df = pd.get_dummies(compas_df, columns=['c_charge_degree']).drop(['c_charge_degree_F'], axis=1)

compas_y = compas_df['two_year_recid'].replace(0, -1)
compas_X = compas_df.drop('two_year_recid', axis=1)

compas_categorical_features = [6, 7, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24]
columns = compas_X.columns
compas_categorical_names = [columns[i] for i in compas_categorical_features] 


/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:8: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  


# Adult dataset

In [21]:
data_name = "adult"
data_file = os.path.join(data_dir, '%s.csv' % data_name)
## load and process data
adult_df = pd.read_csv(data_file).reset_index(drop=True)
adult_df.columns = ['age','workclass','fnlwgt','education','education-num','marital-status','occupation','relationship','race','sex',\
                                          'capital-gain','capital-loss','hours-per-week','native-country','label']

cols_with_missing_values = []
for col in adult_df.columns:
    if len(np.where(adult_df[col].values == '?')[0]) >= 1 or adult_df[col].isnull().values.any():
        cols_with_missing_values.append(col)    

adult_df = adult_df.drop(cols_with_missing_values, axis=1)

adult_df['Married'] = adult_df.apply(lambda row: 1 if 'Married' in row['marital-status'] else 0, axis=1)
adult_df['Widowed'] = adult_df.apply(lambda row: 1 if 'Widowed' in row['marital-status'] else 0, axis=1)
adult_df['NeverMarried'] = adult_df.apply(lambda row: 1 if 'Never-married' in row['marital-status'] else 0, axis=1)

adult_df['workclass_gov'] = adult_df.apply(lambda row: 1 if 'gov' in row['workclass'] else 0, axis=1)
adult_df['workclass_private'] = adult_df.apply(lambda row: 1 if 'Private' in row['workclass'] else 0, axis=1)
adult_df['workclass_self-emp'] = adult_df.apply(lambda row: 1 if 'Self-emp' in row['workclass'] else 0, axis=1)
# adult_df['workclass_never-worked'] = adult_df.apply(lambda row: 1 if 'Never-worked' in row['workclass'] else 0, axis=1)

adult_df['White'] = adult_df.apply(lambda row: 1 if 'White' in row['race'] else 0, axis=1)

# adult_df = pd.get_dummies(adult_df, columns=['race'])
adult_df = pd.get_dummies(adult_df, columns=['sex'])

adult_df = adult_df.drop(['education', 'occupation', 'native-country', \
                          'relationship'], axis=1)
adult_df = adult_df.drop(['sex_ Female', 'race'], axis=1)
adult_df = adult_df.drop(['marital-status', 'workclass', 'fnlwgt'], axis=1)

adult_df.columns = adult_df.columns.str.replace(' ', '')

adult_X = adult_df.drop('label', axis=1)
adult_y = adult_df['label'].replace(' <=50K', -1)
adult_y = adult_y.replace(' >50K', 1)

for col in adult_X.columns:
    print(col)
    print(adult_X[col].value_counts())

adult_categorical_features = [5, 6, 7, 8, 9, 10, 11, 12]
columns = adult_X.columns
print(columns)
adult_categorical_names = [columns[i] for i in adult_categorical_features] 


/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:9: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  if __name__ == '__main__':


age
36    898
31    888
34    886
23    877
35    876
     ... 
83      6
85      3
88      3
87      1
86      1
Name: age, Length: 73, dtype: int64
education-num
9     10501
10     7291
13     5354
14     1723
11     1382
7      1175
12     1067
6       933
4       646
15      576
5       514
8       433
16      413
3       333
2       168
1        51
Name: education-num, dtype: int64
capital-gain
0        29849
15024      347
7688       284
7298       246
99999      159
         ...  
4931         1
1455         1
6097         1
22040        1
1111         1
Name: capital-gain, Length: 119, dtype: int64
capital-loss
0       31041
1902      202
1977      168
1887      159
1848       51
        ...  
1411        1
1539        1
2472        1
1944        1
2201        1
Name: capital-loss, Length: 92, dtype: int64
hours-per-week
40    15216
50     2819
45     1824
60     1475
35     1297
      ...  
92        1
94        1
87        1
74        1
82        1
Name: hours-per-week, Lengt

Make the data not ohe.

In [22]:
#need the data for recourse and lime to be NOT one-hot-encoded and to be numerical
#need the data for the classifier to be one hot encoded

# german_df['YearsAtCurrentJob_lt_1'] = german_df['YearsAtCurrentJob_lt_1'].replace(1, 'lt_1')
# german_df['YearsAtCurrentJob'] = german_df['YearsAtCurrentJob_lt_1']
# german_df['YearsAtCurrentJob_geq_4'] = german_df['YearsAtCurrentJob_geq_4'].replace(1, 'geq_4')
# german_df['YearsAtCurrentJob'] = german_df.apply(lambda row: 'geq_4' if row['YearsAtCurrentJob_geq_4'] == 'geq_4' else row['YearsAtCurrentJob'], axis=1)
# german_df['YearsAtCurrentJob'] = german_df['YearsAtCurrentJob_lt_1'].replace(0, 'bet_1_4')
# german_df = german_df.drop(['YearsAtCurrentJob_lt_1', 'YearsAtCurrentJob_geq_4'], axis=1)

# german_df['CheckingAccountBalance_geq_0'] = german_df['CheckingAccountBalance_geq_0'].replace(1, 'geq_0')
# german_df['CheckingAccountBalance_geq_200'] = german_df['CheckingAccountBalance_geq_200'].replace(1, 'geq_200')
# german_df['CheckingAccountBalance'] = german_df['CheckingAccountBalance_geq_0']
# german_df['CheckingAccountBalance'] = german_df.apply(lambda row: 'geq_200' if row['CheckingAccountBalance_geq_200'] == 'geq_200' else row['CheckingAccountBalance'], axis=1)
# german_df['CheckingAccountBalance'] = german_df['CheckingAccountBalance'].replace('geq_0', '0_200')
# german_df = german_df.drop(['CheckingAccountBalance_geq_0', 'CheckingAccountBalance_geq_200'], axis=1)

# german_df['SavingsAccountBalance_geq_100'] = german_df['SavingsAccountBalance_geq_100'].replace(1, '100_500')
# german_df['SavingsAccountBalance_geq_500'] = german_df['SavingsAccountBalance_geq_500'].replace(1, 'geq_500')
# german_df['SavingsAccountBalance'] = german_df['SavingsAccountBalance_geq_100']
# german_df['SavingsAccountBalance'] = german_df.apply(lambda row: 'geq_500' if row['SavingsAccountBalance_geq_500'] == 'geq_500' else row['SavingsAccountBalance'], axis=1)
# german_df['SavingsAccountBalance'] = german_df['SavingsAccountBalance'].replace('0', 'lt_100')
# german_df = german_df.drop(['SavingsAccountBalance_geq_100', 'SavingsAccountBalance_geq_500'], axis=1)
# display(german_df)


In [23]:
pd.set_option('display.max_columns', None)
# display(X)
# display(y)

In [24]:
# msk = np.random.rand(len(X)) < 0.8
# train = X[msk]
# test = X[~msk]

# train_y = y[msk]
# test_y = y[~msk]

Currently, no immutable features.

# Train model

Ok great, now let's get into the meat of it. Let's train up a model as see what recourse exists.

# Generate Recourse

First, let's score everyone using our model. Now, let's say that we will give loans to anyone with a greater than a $80\%$ chance of paying it back

In [25]:
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import Ridge
from IPython.display import HTML
import time


# start by randomly picking an action for each feature
def sample_with_actions(instance, actions, num_samples, ordered_feature_names):
    num_features = len(ordered_feature_names)
    sampled_data = np.zeros((num_samples, num_features))
    sampled_data[0, :] = instance
    
    len_actions = [len(actions[feat]) for feat in ordered_feature_names]
    ordered_actions = [actions[feat] for feat in ordered_feature_names]
    
#     print("instance: ", instance)
#     print("actions: ", ordered_actions)
#     print("len_actions; ", len_actions)
#     print("ordered_actions: ", ordered_actions)
    
    # max number of actions
    max_actions = len(actions[max(actions, key=lambda feat:len(actions[feat]))])
            
#     print(len_actions)    
        
    for s in range(1, num_samples):
        sampled_actions = [ordered_actions[i][np.random.choice(x)] for i, x in enumerate(len_actions)]
#         print("sampled_actions: ", sampled_actions)
        sampled_data[s, :] = instance + sampled_actions
#         print("sampled_actions: ", sampled_actions)
        
    return sampled_data

def convert_binary_categorical_coefficients(exp_list):
    cleaned_exp_dict = {}
    for (feat, coeff) in exp_list:
        if "=" in feat:
            original_feat, val = feat.split("=")
            int_val = int(val)
            if int_val == 1:
                cleaned_exp_dict[original_feat] = coeff
            else:
                cleaned_exp_dict[original_feat] = -1 * coeff
        else:
            cleaned_exp_dict[feat] = coeff
    return cleaned_exp_dict

# scaled_X = (X - explainer.scaler.mean_) / explainer.scaler.scale_

In [31]:
from functools import partial
from sklearn.model_selection import train_test_split

sys.path.append(os.path.dirname(os.getcwd()))
sys.path.append("../..")
    
# import MAPLE.MAPLE
from MAPLE.Code.Misc import load_normalize_data, unpack_coefs
from MAPLE.Code.MAPLE import MAPLE

def get_nonzero_actions(feature_names, action):
    action_dict = {}
    for feat_idx, feat_name in enumerate(feature_names):
        action_for_feat = action[feat_idx]
        if action_for_feat != 0:
            action_dict[feat_name] = action_for_feat
    return action_dict

# # X_train is unnormalized
# def find_testing_interventions(x, model, X_train, binary_categorical_features, num_divisions = 100):
#     interventions = [[]] * len(X_train.columns)
#     original_pred = model
#     for feat_idx, feat in X_train.columns:
#         if feat_idx not in binary_categorical_features:
#             feat_min = X_train[feat].min()
#             feat_max = X_train[feat].max()
#             increment = (feat_max - feat_min) / 100
#         else:
#             x[feat_idx] = 
        
        
    
    
    
def get_lime_coefficients(lime_explainer, x, model, num_features, num_samples, columns, true_index):
    exp = lime_explainer.explain_instance(x, model.predict_proba, num_features = num_features, num_samples = num_samples)
    local_pred = exp.local_pred

    cleaned_exp_dict = convert_binary_categorical_coefficients(exp.as_list())

    coefficients = [None] * num_features
    
    for j, col in enumerate(columns):
        coefficients[j] = cleaned_exp_dict[col]

    intercept = exp.intercept[true_index]

    x_shift = np.array(lime_explainer.scaler.mean_)
    x_scale = np.array(lime_explainer.scaler.scale_)
    w = coefficients / x_scale
    b = intercept - np.sum(w * x_shift) - 0.5
    
    discrete_yss = (exp.yss[:, true_index] > 0.5).astype(int)
    discrete_sampled_preds = (exp.all_sampled_preds > 0.5).astype(int)
    
    num_pos_yss = (np.count_nonzero(discrete_yss == 1))
    num_neg_yss = (np.count_nonzero(discrete_yss == 0))
    
    num_accurate_preds = np.count_nonzero(discrete_yss == discrete_sampled_preds)
    accuracy_sampled = num_accurate_preds/len(discrete_yss)
    
    return w, b, local_pred, accuracy_sampled

def get_maple_coefficients(maple_explainer, x, mean, std):
    e_maple = maple_explainer.explain(x)
    coefs_maple = e_maple["coefs"][1:]
    intercept_maple = e_maple["coefs"][0]
    w = coefs_maple / mean
    b = intercept_maple - np.sum(w * std) - 0.5
        
    num_pos_yss = (np.count_nonzero(e_maple['selected_sampled_yss'] == 1))
    num_neg_yss = (np.count_nonzero(e_maple['selected_sampled_yss'] == 0))
    
    num_accurate_preds = np.count_nonzero(e_maple['selected_sampled_yss'] == e_maple['selected_sampled_preds'])   
    accuracy_sampled = num_accurate_preds/len(e_maple['selected_sampled_preds'])
    local_pred = e_maple['pred']    
    
    return w, b, local_pred, accuracy_sampled

# assumes data is properly formatted
def calculate_recourse_accuracy(model, data, categorical_features, categorical_names, num_samples = 5000, kernel_width = 1, explanation_type = 'lime', lime_sample_around_instance = False):
    
    X_train = data['X_train']
    y_train = data['y_train']
    
    X_val = data['X_val']
    y_val = data['y_val']
    
    X_test = data['X_test']
    y_test = data['y_test']
    
    print("TRAIN LABEL SPLIT: ")
    print(y_train.value_counts())
    
    print("validation score: ", model.score(X_val, y_val))
    
#     if len(X) >= 100:
#         sampled_df = np.random.choice(np.arange(len(X)), 100, False)
#         sampled_X = X.iloc[sampled_df]
        
#     for col in sampled_X.columns:
#         print(col)
#         print(sampled_X[col].value_counts())
    
    classes = model.classes_
    true_index = list(classes).index(1)
    
    scores = pd.Series(model.predict_proba(X_test)[:, true_index])
    discrete_scores = pd.Series(model.predict(X_test))
        
#     recourses = [None] * len(scores)
    recourses = []

    total_recourses = 0
    total_actual_recourses = 0
    error_instances = 0
    
    num_actiongrid_regressor_agree = 0
    num_lime_agree = 0
    num_sampled_total = 0
    
    print("NUM SAMPLES: ", num_samples)
    print("KERNEL WIDTH: ", kernel_width)
    
    print("num unique preds: ", np.unique(discrete_scores, axis=0).shape[0])
    
    
    print("TEST LABEL SPLIT: ")

    print(discrete_scores.value_counts())

    # class_names have to be ordered according to what the classifier is using
    # need to specify which features are categorical for lime
    lime_explainer = lime_tabular.LimeTabularExplainer(X_train.values, categorical_features=categorical_features, 
                                                       categorical_names=categorical_names, \
                                                       feature_names=X_train.columns, class_names=classes, \
                                                       discretize_continuous=False, kernel_width = kernel_width, \
                                                       sample_around_instance = lime_sample_around_instance)
    
    train_stddev = X_train[X_train.columns[:]].std()
    train_mean = X_train[X_train.columns[:]].mean()
    
    # Normalize to have mean 0 and variance 1
    norm_X_train = (X_train - train_mean) / train_stddev
    norm_X_val = (X_val - train_mean) / train_stddev
    norm_X_test = (X_test - train_mean) / train_stddev
    
    pred_train = model.predict_proba(X_train)[:, true_index]
    pred_val = model.predict_proba(X_val)[:, true_index]
    
    maple_explainer = MAPLE(norm_X_train, pred_train, norm_X_val, pred_val)
    
    action_set = ActionSet(X = X_train)
    display(action_set)

#     for col in X.columns:
#         print(X[col].value_counts())
        
    start_time = time.time()
    num_neg_test_preds = 0
        
    for i, dn in enumerate(scores): #scores is for X_test specifically
        if dn > 0.5:
            continue
        num_neg_test_preds += 1
        if i % 25 == 0:
            print("\n", i, " out of ", len(scores))
        if i % 100 == 0:
            print("time elapsed: ", (time.time() - start_time) / 60, " minutes")
            start_time = time.time()
        x = X_test.values[i]

        num_features = len(x)
        
        if explanation_type == "lime":
            w, b, local_pred, accuracy_sampled = get_lime_coefficients(lime_explainer, x, model, num_features, num_samples, X_train.columns, true_index)
            
        elif explanation_type == "maple":
            w, b, local_pred, accuracy_sampled = get_maple_coefficients(maple_explainer, x, train_mean, train_stddev)
        
        model_pred = (model.predict_proba([x])[0][true_index])
        
        action_set.align(coefficients=w)
        fb = Flipset(x = x, action_set = action_set, coefficients = w, intercept = b)
        
        
        try:
            print("populating lime...")
            fb = fb.populate(enumeration_type = 'distinct_subsets', total_items = 20)
            actions = fb._builder.actions
            
#             print(actions)
                
#             sampled = sample_with_actions(x, actions, 10000, columns)
#             prob_labels = model.predict_proba(sampled)
#             print("sampled: ", sampled)
#             labels = model.predict(sampled)
            
#             distance_metric='euclidean'
#             distances = sklearn.metrics.pairwise_distances(
#                 sampled,
#                 sampled[0].reshape(1, -1),
#                 metric=distance_metric).ravel()
            
#             kernel = None
#             if kernel is None:
#                 def kernel(d, kernel_width):
#                     return np.sqrt(np.exp(-(d ** 2) / kernel_width ** 2))
                
#             kernel_fn = partial(kernel, kernel_width=kernel_width)
#             weights = kernel_fn(distances)
            
            
# #             actiongrid_regressor = Ridge(alpha=1, fit_intercept=True)
# #             actiongrid_regressor.fit(sampled,
# #                            prob_labels, sample_weight = weights) # change weights
    
    
#             actiongrid_regressor = LogisticRegression()
#             print(sampled.shape)
#             print(labels)
#             print("num unique sampled: ", np.unique(sampled, axis=0).shape[0])
#             print("num unique preds for sampled: ", np.unique(labels, axis=0).shape[0])
#             actiongrid_regressor.fit(sampled, labels, sample_weight = weights)

#             print(actiongrid_regressor.coef_)
#             w = actiongrid_regressor.coef_[0]
#             print(w)
#             b = actiongrid_regressor.intercept_[0] - 0.5
#             print(b)
#             actiongrid_regressor_predicted = actiongrid_regressor.predict(sampled)
            
            
#             new_fb = Flipset(x = x, action_set = action_set, coefficients = w, intercept = b)
#             print("populating new...")
#             new_fb = new_fb.populate(enumeration_type = 'distinct_subsets', total_items = 10)
            
#             # lime accuracy on sampled actiongrid points
#             lime_regressor_predicted= np.dot(w, sampled) + b
            
#             num_actiongrid_regressor_agree += np.sum(labels == actiongrid_regressor_predicted)
#             num_lime_agree += np.sum(labels == lime_regressor_predicted)
            
#             num_sampled_total += len(labels)
             
            error = False
            
            returned_actions = [result['actions'] for result in fb.items]

        except ValueError as e:
            print("excepting...")
            print(e)
            print("coeffs from error: ", w)
            error_instances += 1
            error = True
            
            returned_actions = []

        recourse = {}
        recourse['idx'] = i
        recourse['instance'] = x
        recourse['model_prob'] = model_pred
        recourse['local_prob'] = None
        recourse['model_pred'] = 1 if model_pred >= 0.5 else -1
        recourse['local_pred'] = local_pred

        recourse['scaled_coeff'] = w
        recourse['scaled_intercept'] = b
        recourse['actions'] = returned_actions
        recourse['error_solving'] = error
#         recourses[i] = recourse
        recourses.append(recourse)
    
        print_coefs = False
                
        columns = X_train.columns
        for action in returned_actions:
            new_x = (x + action)
            old_pred = recourse['model_pred']
            new_pred = model.predict(new_x.reshape(1, -1))[0]

            new_lime_pred = 1 if np.dot(w, new_x) + b >= 0.0 else -1
            total_recourses += 1

            if old_pred != new_pred:
                print(get_nonzero_actions(columns, action))
                total_actual_recourses += 1
                print_coefs = True
        
        print("model_pred: ", recourse['model_pred']) 
        print("local_pred: ", recourse['local_pred'])
        print("intercept: ", b)
#         if print_coefs:
#             print("model coefs: ")
#             for feat_idx, feat in enumerate(columns):
#                 print(feat, "coef: ", w[feat_idx] * 100, "; original feat: ", x[feat_idx])
            
    total_errors = [1 for rec in recourses if (rec['error_solving'] == True)]            
    
    print("num_neg_test_preds: ", num_neg_test_preds, " out of ", len(scores), " = ", round(num_neg_test_preds/len(scores), 2))
    print("recourse accuracy: ", round(total_actual_recourses/total_recourses, 2), "; total instances with recourses found: ", total_recourses/20)
    print("number of errors: ", sum(total_errors), "; percent of total instances: ", round(sum(total_errors)/len(recourses), 2))
#     print("LIME avg fidelity of explanations: ", lime_regressor_predicted/num_sampled_total)
#     print("NEW METHOD avg fidelity of explanations: ", num_actiongrid_regressor_agree/num_sampled_total)
    
    

In [27]:
def get_data(X, y):
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33)
    X_test, X_val, y_test, y_val = train_test_split(X_test, y_test, test_size=0.5)
    
    data = {
        'X_train': X_train,
        'y_train': y_train,

        'X_val': X_val,
        'y_val': y_val,

        'X_test': X_test,
        'y_test': y_test
    }
    
    return data    

# Experiments

## German

In [28]:
from sklearn.neural_network import MLPClassifier
from sklearn.ensemble import RandomForestClassifier
# import lime.explanation
# import lime.lime_tabular

sys.path.append(os.path.dirname(os.getcwd()))
sys.path.append("../lime_experiments")
    
import explanation
import lime_tabular
import lime_base

import importlib
importlib.reload(explanation)
importlib.reload(lime_tabular)
importlib.reload(lime_base)

<module 'lime_base' from '../lime_experiments/lime_base.py'>

In [29]:
# NEURAL NETWORK

german_nn = MLPClassifier()

german_data = get_data(german_X, german_y)

german_nn.fit(german_data['X_train'], german_data['y_train']) 
print("validation score: ", round(german_nn.score(german_data['X_val'], german_data['y_val']), 2))
test_preds = pd.Series(german_nn.predict(german_data['X_test']))
print("test predictions split: ")
print(test_preds.value_counts())

# german_rf = RandomForestClassifier(n_estimators=40)


validation score:  0.6
test predictions split: 
 1    102
-1     63
dtype: int64


In [ ]:
calculate_recourse_accuracy(german_nn, german_data, german_categorical_features, german_categorical_names, lime_sample_around_instance = False)



In [ ]:
calculate_recourse_accuracy(german_nn, german_data, german_categorical_features, german_categorical_names, lime_sample_around_instance = True)


In [32]:
calculate_recourse_accuracy(german_nn, german_data, german_categorical_features, german_categorical_names, explanation_type = "maple")


TRAIN LABEL SPLIT: 
 1    466
-1    204
Name: GoodCustomer, dtype: int64
validation score:  0.6
NUM SAMPLES:  5000
KERNEL WIDTH:  1
num unique preds:  2
TEST LABEL SPLIT: 
 1    102
-1     63
dtype: int64


+---------------------------------+---------------+---------+------------+----------------+----------------+-----------+-----------+-----------+-------+---------+
|                            name | variable type | mutable | actionable | step direction | flip direction | grid size | step type | step size |    lb |      ub |
+---------------------------------+---------------+---------+------------+----------------+----------------+-----------+-----------+-----------+-------+---------+
|                   ForeignWorker | <class 'int'> |    True |       True |              0 |            nan |         2 |  relative |      0.01 |   0.0 |     1.0 |
|                          Single | <class 'int'> |    True |       True |              0 |            nan |         2 |  relative |      0.01 |   0.0 |     1.0 |
|                             Age | <class 'int'> |    True |       True |              0 |            nan |        49 |  relative |      0.01 |  20.0 |    68.0 |
|                    L

1.0
populating lime...
recovered all minimum-cost items
obtained 0 items in 0.1 seconds
model_pred:  -1
local_pred:  [-339.59131147]
intercept:  -0.18292487582577377
1.0
populating lime...
obtained 20 items in 0.8 seconds
{'Age': 14.0, 'LoanAmount': -2855.0}
{'Age': 13.0, 'LoanDuration': 1.0, 'LoanAmount': -2855.0}
{'LoanDuration': 15.0, 'LoanAmount': -2855.0}
{'Age': 12.0, 'LoanAmount': -2855.0, 'NumberOfOtherLoansAtBank': 1.0}
{'Age': 11.0, 'LoanDuration': 1.0, 'LoanAmount': -2855.0, 'NumberOfOtherLoansAtBank': 1.0}
{'LoanDuration': 13.0, 'LoanAmount': -2855.0, 'NumberOfOtherLoansAtBank': 1.0}
{'Age': 39.0, 'LoanDuration': 24.0}
{'Age': 39.0, 'LoanDuration': 23.0, 'NumberOfOtherLoansAtBank': 1.0}
model_pred:  -1
local_pred:  [-529.47930434]
intercept:  -0.15900426588919464
1.0
populating lime...
obtained 20 items in 0.8 seconds
{'Age': 40.0, 'LoanDuration': 44.0, 'LoanAmount': -247.0, 'NoCurrentLoan': -1.0}
{'Age': 42.0, 'LoanDuration': 47.0, 'NoCurrentLoan': -1.0}
{'Age': 39.0, 'Loa

obtained 20 items in 1.1 seconds
{'Age': 12.0, 'LoanDuration': 29.0, 'LoanAmount': -1164.0}
{'Age': 10.0, 'LoanDuration': 29.0, 'LoanAmount': -1164.0, 'YearsAtCurrentHome': 1.0}
{'LoanDuration': 40.0, 'LoanAmount': -1164.0, 'YearsAtCurrentHome': 1.0}
{'LoanDuration': 42.0, 'LoanAmount': -1164.0}
{'Age': 22.0, 'LoanDuration': 39.0}
{'Age': 22.0, 'LoanDuration': 37.0, 'YearsAtCurrentHome': 1.0}
{'Age': 11.0, 'LoanDuration': 29.0, 'LoanAmount': -1164.0, 'NumberOfOtherLoansAtBank': 1.0}
{'Age': 10.0, 'LoanDuration': 28.0, 'LoanAmount': -1164.0, 'YearsAtCurrentHome': 1.0, 'NumberOfOtherLoansAtBank': 1.0}
{'LoanDuration': 41.0, 'LoanAmount': -1164.0, 'NumberOfOtherLoansAtBank': 1.0}
{'LoanDuration': 39.0, 'LoanAmount': -1164.0, 'YearsAtCurrentHome': 1.0, 'NumberOfOtherLoansAtBank': 1.0}
{'Age': 21.0, 'LoanDuration': 39.0, 'NumberOfOtherLoansAtBank': 1.0}
{'Age': 22.0, 'LoanDuration': 36.0, 'YearsAtCurrentHome': 1.0, 'NumberOfOtherLoansAtBank': 1.0}
{'ForeignWorker': 1.0}
{'ForeignWorker': 1.

obtained 20 items in 0.9 seconds
{'LoanAmount': -6147.0, 'NoCurrentLoan': -1.0}
model_pred:  -1
local_pred:  [-1233.41073247]
intercept:  -0.1659794936283962
1.0
populating lime...
obtained 20 items in 0.9 seconds
{'Age': 12.0, 'LoanDuration': 6.0, 'LoanAmount': -2964.0}
{'Age': 18.0, 'LoanAmount': -2964.0}
{'LoanDuration': 19.0, 'LoanAmount': -2964.0}
{'Age': 11.0, 'LoanDuration': 6.0, 'LoanAmount': -2964.0, 'YearsAtCurrentHome': 1.0}
{'Age': 17.0, 'LoanAmount': -2964.0, 'YearsAtCurrentHome': 1.0}
{'LoanDuration': 18.0, 'LoanAmount': -2964.0, 'YearsAtCurrentHome': 1.0}
{'Age': 15.0, 'LoanDuration': 2.0, 'LoanAmount': -2964.0, 'NumberOfOtherLoansAtBank': 1.0}
{'Age': 17.0, 'LoanAmount': -2964.0, 'NumberOfOtherLoansAtBank': 1.0}
{'LoanDuration': 18.0, 'LoanAmount': -2964.0, 'NumberOfOtherLoansAtBank': 1.0}
{'Age': 10.0, 'LoanDuration': 6.0, 'LoanAmount': -2964.0, 'YearsAtCurrentHome': 1.0, 'NumberOfOtherLoansAtBank': 1.0}
{'Age': 16.0, 'LoanAmount': -2964.0, 'YearsAtCurrentHome': 1.0, '

obtained 20 items in 0.8 seconds
{'Age': 22.0, 'LoanAmount': -2856.0}
{'Age': 13.0, 'LoanDuration': 10.0, 'LoanAmount': -2856.0}
{'Age': 21.0, 'LoanAmount': -2856.0, 'LoanRateAsPercentOfIncome': -1.0}
{'Age': 20.0, 'LoanDuration': 1.0, 'LoanAmount': -2856.0, 'LoanRateAsPercentOfIncome': -1.0}
{'Age': 21.0, 'LoanAmount': -2856.0, 'YearsAtCurrentHome': 1.0}
{'Age': 20.0, 'LoanDuration': 1.0, 'LoanAmount': -2856.0, 'YearsAtCurrentHome': 1.0}
{'Age': 20.0, 'LoanAmount': -2856.0, 'LoanRateAsPercentOfIncome': -1.0, 'YearsAtCurrentHome': 1.0}
{'Age': 13.0, 'LoanDuration': 7.0, 'LoanAmount': -2856.0, 'LoanRateAsPercentOfIncome': -1.0, 'YearsAtCurrentHome': 1.0}
{'LoanDuration': 24.0, 'LoanAmount': -2856.0}
{'LoanDuration': 23.0, 'LoanAmount': -2856.0, 'LoanRateAsPercentOfIncome': -1.0}
{'LoanDuration': 23.0, 'LoanAmount': -2856.0, 'YearsAtCurrentHome': 1.0}
{'LoanDuration': 22.0, 'LoanAmount': -2856.0, 'LoanRateAsPercentOfIncome': -1.0, 'YearsAtCurrentHome': 1.0}
{'Age': 21.0, 'LoanAmount': -2

ApplicationError: Solver (cbc) did not exit normally

In [ ]:
from sklearn.neural_network import MLPClassifier
from sklearn.ensemble import RandomForestClassifier

# NEURAL NETWORK

german_rf = RandomForestClassifier()

german_rf.fit(german_data['X_train'], german_data['y_train']) 
print("validation score: ", round(german_rf.score(german_data['X_val'], german_data['y_val']), 2))
test_preds = pd.Series(german_rf.predict(german_data['X_test']))
print("test predictions split: ")
print(test_preds.value_counts())

calculate_recourse_accuracy(german_rf, german_data, german_categorical_features, german_categorical_names)


In [ ]:
calculate_recourse_accuracy(german_rf, german_data, german_categorical_features, german_categorical_names, explanation_type = "maple")


## COMPAS

In [ ]:
from sklearn.neural_network import MLPClassifier
from sklearn.ensemble import RandomForestClassifier
import lime.explanation
import lime.lime_tabular

# NEURAL NETWORK

compas_nn = MLPClassifier()

compas_data = get_data(compas_X, compas_y)

compas_nn.fit(compas_data['X_train'], compas_data['y_train']) 
print("validation score: ", round(compas_nn.score(compas_data['X_val'], compas_data['y_val']), 2))
test_preds = pd.Series(compas_nn.predict(compas_data['X_test']))
print("test predictions split: ")
print(test_preds.value_counts())

In [ ]:
print("COMPAS")
calculate_recourse_accuracy(compas_nn, compas_data, compas_categorical_features, compas_categorical_names)


In [ ]:
calculate_recourse_accuracy(compas_nn, compas_data, compas_categorical_features, compas_categorical_names, explanation_type = "maple")


## ADULT

In [ ]:
print("ADULT")
calculate_recourse_accuracy(nn, adult_X, adult_y, adult_categorical_features, adult_categorical_names)


In [ ]:
print("GERMAN")
calculate_recourse_accuracy(rf, german_X, german_y, german_categorical_features, german_categorical_names)

In [ ]:
print("COMPAS")
calculate_recourse_accuracy(rf, compas_X, compas_y, compas_categorical_features, compas_categorical_names)


In [ ]:
print("ADULT")
display(adult_X)

calculate_recourse_accuracy(rf, adult_X, adult_y, adult_categorical_features, adult_categorical_names)


You can switch optimizers if you don't have CPLEX by setting `optimizer="cbc"`. 

A quick note: Our decision boundary is by default 0. We shift this by tweaking the intercept. Since we used Logistic Regression, we use the trick above to do that. In future iterations, we will provide a more elegant way of doing this.

In [ ]:
output_1 = rb.fit()
output_1

all_info = rb.populate()
print(all_info)

Ok, great, we have a solution! This individual has recourse. The total cost of all the actions needed to flip their prediction is the first thing of interest to us. It costs this person $.21$, meaning that the sum of percentile shifts across this person's features is $.21$. That's quite a lot. Imagine having to shift that much relative to a population? Let's check out what this means in terms of actions:

In [ ]:
# pd.Series(output_1['actions'], index=X.columns).to_frame('Actions')
actions = [x['actions'] for x in all_info]
actions_df = pd.DataFrame(data=actions).transpose().set_index(X.columns)
person = (pd.Series(x, index=X.columns))
print(person)
display(actions_df)

Ok, so let's read this. 

* `SavingsAccountBalance_geq_100`$=1$, for example. This was a binary feature, so it can only be $1$. This also means that we're enouraging this person to increase their savings. 
* `LoanDuration`$=20$. This, if we recall, was the number of months of loan. This means we're encouraging this person to reapply but specify that their loan repayment period is 20 months shorter.

Let's check if these two actions make sense in the context of this person:

In [ ]:
X.loc[denied_individuals[0]].to_frame("Original Features")

Ok, this person originally applied with no savings and with a 4-year repayment period. So asking them to get savings and decrease their loan repayment period by $20$ months make sense as actions.

(Let's leave aside the question of mutually exclusive features (eg. `SavingsAccountBalance_geq_100` $=0$, `SavingsAccountBalance_geq_500`$=1$). We'll get back to that in later releases.)

Let's close by noting some things:

* Immutable features are __not__ changed. That's good. That's recourse.
* The changes make sense, at least directionally. We'd encourage this person to get a gaurantor, to decrease their loan amount, and to decrease their loan period, among other changes.

Yes, these might be hard for someone. They might have other reasons for immutability that we're not considering. Maybe they _need_ that amount and cannot change. Ok, let's express that:

In [ ]:
action_set['LoanAmount'].mutable=False

In [ ]:
x = X.values[denied_individuals[0]]

p = .8
rb = RecourseBuilder(
      optimizer="cbc",
      coefficients=coefficients,
      intercept=intercept- (np.log(p / (1. - p))),
      action_set=action_set,
      x=x
)

In [ ]:
output_2 = rb.fit()
output_2

Ok, so their total cost actually didn't change, which is nice. Let's take a look at their new action set:

In [ ]:
pd.Series(output_2['actions'], index=X.columns).to_frame("New Actions")

Ok, by decreasing their repayment period by a bit more and changing some other features, this person can still ask for the same amount. That's good.

The magical thing about both of these action sets is that this person, if they do this, _will_ qualify for a loan. Let's check that:

In [ ]:
clf.predict_proba([X.loc[denied_individuals[0]] + pd.Series(output_1['actions'], index=X.columns)])[:, 1]

In [ ]:
clf.predict_proba([X.loc[denied_individuals[0]] + pd.Series(output_2['actions'], index=X.columns)])[:, 1]

And there we have it. By making these tweaks, this person has two ways to get over the $.8$ threshold that we've set. This period can now get approved under this model.